In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import random
from typing import List, Union
from joblib import Parallel, delayed

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

import datetime
import matplotlib.pyplot as plt

In [ ]:
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수(-1 예측 포함)
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    rle = ' '.join(str(x) for x in runs)
    return rle if rle != '' else -1

In [ ]:
custom_transform = {
    'train':A.Compose([
        # A.augmentations.crops.transforms.RandomCrop(224,224,p=1.0),
        # A.Normalize(),
        # A.OneOf([
        #     A.augmentations.crops.transforms.RandomCrop(224,224,p=0.3),
        #     A.Resize(224, 224, p = 0.7),
        # ], p=1.0),
        A.augmentations.crops.transforms.CropNonEmptyMaskIfExists(224,224,[0]),
        ToTensorV2()
    ]),
    'valid':A.Compose([
        A.augmentations.crops.transforms.CenterCrop(224,224,p=1.0),
        A.Normalize(),
        ToTensorV2()
    ]),
    'test': A.Compose([
        A.Normalize(),
        ToTensorV2()
    ]),
}

In [ ]:
idx = 0 # train에서 보고 싶은 index

train = pd.read_csv('../open (1)/train.csv') # 경로 주의
image = cv2.imread(f'../open (1)/{train.iloc[idx,1]}') # 경로 주의
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
mask_rle = train.iloc[idx,2]
mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))

In [ ]:
augmented = custom_transform['train'](image=image, mask=mask)
new_image = augmented['image']
new_mask = augmented['mask']
new_image = np.transpose(new_image, (1, 2, 0))

fig = plt.figure(figsize = [15,10])
ax1 = fig.add_subplot(1,4,1)
ax1.imshow(image)
ax1.set_title(f'{image.shape}')
ax2 = fig.add_subplot(1,4,2)
ax2.imshow(mask)
ax2.set_title(f'{mask.shape}')
ax3 = fig.add_subplot(1,4,3)
ax3.imshow(new_image)
ax3.set_title(f'{new_image.shape}')
ax4 = fig.add_subplot(1,4,4)
ax4.imshow(new_mask)
ax4.set_title(f'{new_mask.shape}')